#### Imports

In [1]:
import json
import nltk
from nltk.collocations import *
import datetime as dt
import locale
import spacy
from tqdm import tqdm
import pprint
import pandas as pd
from ast import literal_eval
import os
import gensim
import gensim.corpora as corpora
from pprint import pprint


In [19]:
#!python -m spacy download de_core_news_lg

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
#NOTE: default version doesntwork in colab
#!pip install pyLDAvis==2.1.2

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Read Data

In [5]:
# connect with google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#change cwd
%cd drive/MyDrive/Work/Frontline/data

/content/drive/.shortcut-targets-by-id/1WfnZsqpG1r110J63sMbfS5TpsDOkveiV/data


In [7]:
# paths
FILTERED_PATH="filtered_4_26"

In [8]:
## FOR NOW: USE SMALL DATA SET
df=pd.read_csv("sample.csv")

### Topic Modelling


#### Prepare Data


In [9]:
# custom module
import preprocessing

In [10]:
# Load model
spacy_mod = spacy.load("de_core_news_lg", disable=['ner', 'parser', 'tagger'])

In [11]:
# read custom stopwords

# open list of custom stopwords
custom_stop_words= open("custom_stopwords.txt").read().split()

# add custom stopwords to model
for word in custom_stop_words:
  spacy_mod.Defaults.stop_words.add(word)

In [12]:
# convert corpus to language object
spacy_lang = []
for i, doc in tqdm(df.iterrows()): 
  spacy_lang.append(spacy_mod("".join(doc['text'])))

50it [00:02, 20.37it/s]


In [13]:
# preprocess: remove stopwords
spacy_cleaned = []
for doc in tqdm(spacy_lang): 
    spacy_cleaned.append(preprocessing.preprocess(doc, remove_ent=True))

100%|██████████| 50/50 [00:00<00:00, 1150.44it/s]


#### Topic Analysis

In [14]:
# Create Dictionary
id2word = corpora.Dictionary(spacy_cleaned)

# Create Corpus
texts = spacy_cleaned

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [15]:
# number of topics
num_topics = 5
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,id2word=id2word, num_topics=num_topics)
# Print the key words for each topic
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.015*"gewalt" + 0.007*"frauen" + 0.006*"häuslicher" + 0.006*"frau" + '
  '0.004*"mann" + 0.004*"polizei" + 0.004*"opfer" + 0.003*"fälle" + '
  '0.003*"prozent" + 0.003*"männer"'),
 (1,
  '0.011*"frauen" + 0.011*"gewalt" + 0.010*"frau" + 0.009*"opfer" + '
  '0.005*"mann" + 0.004*"häuslicher" + 0.004*"männer" + 0.004*"polizei" + '
  '0.003*"prozent" + 0.003*"menschen"'),
 (2,
  '0.020*"frauen" + 0.019*"gewalt" + 0.006*"opfer" + 0.006*"frau" + '
  '0.006*"häuslicher" + 0.004*"männer" + 0.003*"fälle" + 0.003*"landkreis" + '
  '0.003*"mann" + 0.003*"prozent"'),
 (3,
  '0.010*"frau" + 0.008*"mann" + 0.008*"frauen" + 0.007*"gewalt" + '
  '0.006*"opfer" + 0.006*"polizei" + 0.004*"männer" + 0.004*"kinder" + '
  '0.003*"häuslicher" + 0.003*"mutter"'),
 (4,
  '0.011*"gewalt" + 0.007*"opfer" + 0.007*"frauen" + 0.007*"frau" + '
  '0.004*"mann" + 0.003*"häuslicher" + 0.003*"polizei" + 0.002*"weißen" + '
  '0.002*"prozent" + 0.002*"leben"')]


In [18]:
import pyLDAvis
import pyLDAvis.gensim
import pickle 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pyLDAvis/_prepare.py:232: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.035536 -0.001159       1        1  33.440230
3     -0.018311 -0.013384       2        1  21.298935
1     -0.001418 -0.006741       3        1  18.706762
0     -0.009873  0.022117       4        1  18.438835
4     -0.005934 -0.000832       5        1   8.115238, topic_info=           Term       Freq      Total Category  logprob  loglift
176       opfer  44.000000  44.000000  Default  30.0000  30.0000
114      gewalt  93.000000  93.000000  Default  29.0000  29.0000
8          frau  53.000000  53.000000  Default  28.0000  28.0000
15         mann  33.000000  33.000000  Default  27.0000  27.0000
20      polizei  25.000000  25.000000  Default  26.0000  26.0000
...         ...        ...        ...      ...      ...      ...
357      kinder   0.874393  16.888477   Topic5  -6.4854  -0.4494
1616       laut   0.793276  11.663339   Topic5  -6.5828  -0.1766
353   häusliche   0.836966  15.844063   Topic5  -6.5292  -0.4293
678      männer   0.924232  25.708311   Topic5  -6.4300  -0.8142
476       fälle   0.803103  17.668614   Topic5  -6.5705  -0.5796

[375 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
463       1  0.657788       aktion
463       2  0.131558       aktion
463       3  0.131558       aktion
266       1  0.551774  akzeptieren
1658      2  0.758885    alarmiert
...     ...       ...          ...
781       2  0.260214        zuvor
781       3  0.260214        zuvor
781       4  0.260214        zuvor
2121      4  0.758289       zählte
2000      4  0.758083      übersah

[550 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 2, 1, 5])

END OF CODE